In [2]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv1D
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.multivariate.pca import PCA
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from math import sqrt
from statsmodels.tsa.api import ExponentialSmoothing
from statsmodels.tsa.holtwinters import SimpleExpSmoothing, Holt
from statsmodels.tsa.api import VAR
import numpy as np
import seaborn as sns

path = "./raw_data/"

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
df1201 = pd.read_excel(path+"Analysisprogram_1201_final.xls", index_col=None, header=0)
df1201 = df1201[9:40]
df1202 = pd.read_excel(path+"Analysisprogram_1202_final.xls", index_col=None, header=0)
df1202 = df1202[9:38]
df1203 = pd.read_excel(path+"Analysisprogram_1203_final.xls", index_col=None, header=0)
df1203 = df1203[9:40]
df1204 = pd.read_excel(path+"Analysisprogram_1204_final.xls", index_col=None, header=0)
df1204 = df1204[9:39]
df1205 = pd.read_excel(path+"Analysisprogram_1205_final.xls", index_col=None, header=0)
df1205 = df1205[9:40]
df1206 = pd.read_excel(path+"Analysisprogram_1206_final.xls", index_col=None, header=0)
df1206 = df1206[9:39]
df1207 = pd.read_excel(path+"Analysisprogram_1207_final.xls", index_col=None, header=0)
df1207 = df1207[9:40]
df1208 = pd.read_excel(path+"Analysisprogram_1208_final.xls", index_col=None, header=0)
df1208 = df1208[9:40]
df1209 = pd.read_excel(path+"Analysisprogram_1209_final.xls", index_col=None, header=0)
df1209 = df1209[9:39]
df1210 = pd.read_excel(path+"Analysisprogram_1210_final.xls", index_col=None, header=0)
df1210 = df1210[9:40]
df1211 = pd.read_excel(path+"Analysisprogram_1211_final.xls", index_col=None, header=0)
df1211 = df1211[9:39]
df1212 = pd.read_excel(path+"Analysisprogram_1212_final.xls", index_col=None, header=0)
df1212 = df1212[9:40]
df1301 = pd.read_excel(path+"Analysisprogram_1301_final.xls", index_col=None, header=0)
df1301 = df1301[9:40]
df1302 = pd.read_excel(path+"Analysisprogram_1302_final.xls", index_col=None, header=0)
df1302 = df1302[9:37]
df1303 = pd.read_excel(path+"Analysisprogram_1303_final.xls", index_col=None, header=0)
df1303 = df1303[9:40]
df1304 = pd.read_excel(path+"Analysisprogram_1304_final.xls", index_col=None, header=0)
df1304 = df1304[9:39]
df1305 = pd.read_excel(path+"Analysisprogram_1305_final.xls", index_col=None, header=0)
df1305 = df1305[9:40]
df1306 = pd.read_excel(path+"Analysisprogram_1306_final.xls", index_col=None, header=0)
df1306 = df1306[9:39]
df1307 = pd.read_excel(path+"Analysisprogram_1307_final.xls", index_col=None, header=0)
df1307 = df1307[9:40]
df1308 = pd.read_excel(path+"Analysisprogram_1308_final.xls", index_col=None, header=0)
df1308 = df1308[9:40]
df1309 = pd.read_excel(path+"Analysisprogram_1309_final.xls", index_col=None, header=0)
df1309 = df1309[9:39]
df1310 = pd.read_excel(path+"Analysisprogram_1310_final.xls", index_col=None, header=0)
df1310 = df1310[9:40]
df1311= pd.read_excel(path+"Analysisprogram_1311_final.xls", index_col=None, header=0)
df1311 = df1311[9:39]
df1312 = pd.read_excel(path+"Analysisprogram_1312_final.xls", index_col=None, header=0)
df1312 = df1312[9:40]
df1401 = pd.read_excel(path+"Analysisprogram_1401_final.xls", index_col=None, header=0)
df1401 = df1401[9:40]
df1402 = pd.read_excel(path+"Analysisprogram_1402_final.xls", index_col=None, header=0)
df1402 = df1402[9:37]
df1403 = pd.read_excel(path+"Analysisprogram_1403_final.xls", index_col=None, header=0)
df1403 = df1403[9:40]
df1404 = pd.read_excel(path+"Analysisprogram_1404_final.xls", index_col=None, header=0)
df1404 = df1404[9:39]

In [ ]:
concatenated_df = pd.concat([df1201,df1202,df1203,df1204,df1205,df1206,df1207,df1208,df1209,df1210,df1211,df1212,
                     df1301,df1302,df1303,df1304,df1305,df1306,df1307,df1308,df1309,df1310,df1311,df1312,
                     df1401,df1402,df1403,df1404 ],ignore_index=True)

In [ ]:
# MAKE DTI OBJECT, USE AS INDEX FOR PREP1DF

concatenated_df.to_csv("prep1df_ver_1.csv")

In [ ]:
prep1df = pd.read_csv("prep1df_renamed.csv")#, index_col=0)#Changed Column Names in Excel
prep1df.rename(columns={'Unnamed: 0':'Date'}, inplace=True)
prep1df.drop(columns=["Day"],inplace=True)
dti = pd.date_range("2012-01-01", periods=851, freq="D")
prep1df.Date = dti
prep1df.set_index('Date', inplace=True)
missing_values = prep1df.isna().sum().sort_values(ascending=False)
missing_values.head(26)

CAKE                                                851
SONIC_TWAS_DDCOD_WEEK                               851
TWAS_DAF_Q2_DAY                                     851
TWAS_CENT_Q3_DAY                                    851
DIG_SLUDGE_DEWATER_FEED                             851
SONIC_TWAS_Q_DAY                                    851
CH4CONTENT_WEEKLY                                   806
TWAS_CENT_VS_DAY                                    484
TWAS_CENT_DS_DAY                                    460
DIG_SLUDGE_DEWATER_PLYMER_CONSUMPTION_3_PER_WEEK    432
TWAS_DAF_VS_DAY                                     347
DIGESTED_SLUDGE_VS_3_PER_WEEK                       338
TPS_VS_DAY                                          333
TWAS_DAF_DS_DAY                                     325
PS_VS_DAY                                           324
DIGESTED_SLUDGE_DS_3_PER_WEEK                       314
TPS_DS_DAY                                          311
PS_DS_DAY                                       

In [ ]:
prep1df

,PS_Q_DAY,PS_DS_DAY,PS_VS_DAY,TPS_Q1_DAY,TPS_DS_DAY,TPS_VS_DAY,TWAS_DAF_QIN_DAY,TWAS_DAF_Q2_DAY,TWAS_DAF_DS_DAY,TWAS_DAF_VS_DAY,TWAS_CENT_Q3_DAY,TWAS_CENT_DS_DAY,TWAS_CENT_VS_DAY,SONIC_TWAS_Q_DAY,SONIC_TWAS_DDCOD_WEEK,DIGESTED_SLUDGE_QOUT_DAY,DIGESTED_SLUDGE_DS_3_PER_WEEK,DIGESTED_SLUDGE_VS_3_PER_WEEK,BIOGAS_PRODUCTION_Q_DAY,CH4CONTENT_WEEKLY,DIG_SLUDGE_DEWATER_FEED,DIG_SLUDGE_DEWATER_FECL3_DAY,DIG_SLUDGE_DEWATER_PLYMER_CONSUMPTION_3_PER_WEEK,DIG_SLUDGE_DEWATER_DS_AFTER_DEWATER_3_PER_WEEK,CAKE
Date,,,,,,,,,,,,,,,,,,,,,,,,,
2012-01-01,1197.0,NaN,NaN,199.5,NaN,NaN,242.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,414,NaN,NaN,5.064,NaN,NaN,5600.0,NaN,32.1,NaN
2012-01-02,1197.0,NaN,NaN,199.5,NaN,NaN,165.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,392,NaN,NaN,5.293,NaN,NaN,2800.0,NaN,32.1,NaN
2012-01-03,1197.0,0.18,60.00,199.5,2.01,89.13,241.5,NaN,5.01,87.25,NaN,NaN,NaN,NaN,NaN,428,2.09,77.85,5.609,NaN,NaN,4200.0,3.865907,32.1,NaN
2012-01-04,1197.0,0.57,81.81,170.0,2.91,91.30,194.0,NaN,5.48,82.17,NaN,2.78,82.92,NaN,NaN,354,1.88,NaN,4.470,NaN,NaN,5600.0,4.399710,31.8,NaN
2012-01-05,1197.0,0.45,72.72,199.5,3.17,91.22,193.5,NaN,5.36,84.78,NaN,3.98,81.73,NaN,NaN,383,2.05,77.41,2.848,NaN,NaN,5600.0,4.509834,31.4,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014-04-26,711.0,NaN,NaN,199.5,NaN,NaN,271.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,470,NaN,NaN,1.824,NaN,NaN,2800.0,NaN,32.0,NaN
2014-04-27,711.0,NaN,NaN,199.5,NaN,NaN,188.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,390,NaN,NaN,1.670,NaN,NaN,1400.0,NaN,30.0,NaN
2014-04-28,993.0,1.90,84.00,199.5,1.9,85.00,185.5,NaN,2.70,82.00,NaN,4.90,81.00,NaN,NaN,392,1.80,76.00,1.613,NaN,NaN,1400.0,NaN,32.0,NaN


In [ ]:
prep1df.drop(['CAKE',
              'SONIC_TWAS_DDCOD_WEEK',
              'DIG_SLUDGE_DEWATER_FEED',
              'SONIC_TWAS_Q_DAY',
              'CH4CONTENT_WEEKLY',
              'TWAS_DAF_Q2_DAY',
              'DIG_SLUDGE_DEWATER_PLYMER_CONSUMPTION_3_PER_WEEK',
              'TWAS_CENT_Q3_DAY',
              
              ], axis=1, inplace=True)

In [ ]:

prep1df.to_csv("better_prepdf.csv")

In [ ]:
prepdf = pd.read_csv('better_prepdf.csv',index_col=0)

filled_df = prepdf.copy()
prepdf.head()

,PS_Q_DAY,PS_DS_DAY,PS_VS_DAY,TPS_Q1_DAY,TPS_DS_DAY,TPS_VS_DAY,TWAS_DAF_QIN_DAY,TWAS_DAF_DS_DAY,TWAS_DAF_VS_DAY,TWAS_CENT_DS_DAY,TWAS_CENT_VS_DAY,DIGESTED_SLUDGE_QOUT_DAY,DIGESTED_SLUDGE_DS_3_PER_WEEK,DIGESTED_SLUDGE_VS_3_PER_WEEK,BIOGAS_PRODUCTION_Q_DAY,DIG_SLUDGE_DEWATER_FECL3_DAY,DIG_SLUDGE_DEWATER_DS_AFTER_DEWATER_3_PER_WEEK
Date,,,,,,,,,,,,,,,,,
2012-01-01,1197.0,NaN,NaN,199.5,NaN,NaN,242.5,NaN,NaN,NaN,NaN,414,NaN,NaN,5.064,5600.0,32.1
2012-01-02,1197.0,NaN,NaN,199.5,NaN,NaN,165.5,NaN,NaN,NaN,NaN,392,NaN,NaN,5.293,2800.0,32.1
2012-01-03,1197.0,0.18,60.00,199.5,2.01,89.13,241.5,5.01,87.25,NaN,NaN,428,2.09,77.85,5.609,4200.0,32.1
2012-01-04,1197.0,0.57,81.81,170.0,2.91,91.30,194.0,5.48,82.17,2.78,82.92,354,1.88,NaN,4.470,5600.0,31.8
2012-01-05,1197.0,0.45,72.72,199.5,3.17,91.22,193.5,5.36,84.78,3.98,81.73,383,2.05,77.41,2.848,5600.0,31.4


In [ ]:
percent_missing = filled_df.isnull().sum() * 100 / len(filled_df)
missing_value_df = pd.DataFrame({'percent_missing': percent_missing})
missing_value_df

,percent_missing
PS_Q_DAY,0.117509
PS_DS_DAY,35.370153
PS_VS_DAY,38.072855
TPS_Q1_DAY,0.117509
TPS_DS_DAY,36.545241
TPS_VS_DAY,39.130435
TWAS_DAF_QIN_DAY,0.000000
TWAS_DAF_DS_DAY,38.190364
TWAS_DAF_VS_DAY,40.775558
TWAS_CENT_DS_DAY,54.054054


In [ ]:
# Drop Columns with sparsity >35%
prepdf_nonpca = prepdf.copy()
prepdf_nonpca.drop(columns=['TWAS_DAF_DS_DAY', 'TWAS_DAF_VS_DAY', 'TWAS_CENT_VS_DAY', 'PS_DS_DAY','TWAS_CENT_DS_DAY','TPS_VS_DAY', 'DIG_SLUDGE_DEWATER_FECL3_DAY', 'DIGESTED_SLUDGE_DS_3_PER_WEEK', 'PS_VS_DAY','TPS_DS_DAY', 'DIGESTED_SLUDGE_VS_3_PER_WEEK' ], inplace=True)
filled_df = prepdf_nonpca.copy()

In [ ]:

prepdf_nonpca = prepdf_nonpca.reset_index()


In [ ]:
# PS_Q_DAY
df_PS_Q_DAY = prepdf_nonpca.drop(columns=['DIG_SLUDGE_DEWATER_DS_AFTER_DEWATER_3_PER_WEEK', 'TPS_Q1_DAY'])

df_PS_Q_DAYWith = df_PS_Q_DAY[pd.isnull(df_PS_Q_DAY['PS_Q_DAY']) == False]
df_PS_Q_DAYWithout = df_PS_Q_DAY[pd.isnull(df_PS_Q_DAY['PS_Q_DAY'])]

variables = ['PS_Q_DAY', 'TWAS_DAF_QIN_DAY', 'DIGESTED_SLUDGE_QOUT_DAY', 'BIOGAS_PRODUCTION_Q_DAY']

independentVariables = ['TWAS_DAF_QIN_DAY', 'DIGESTED_SLUDGE_QOUT_DAY', 'BIOGAS_PRODUCTION_Q_DAY']

rfModel_PS_Q_DAY = RandomForestRegressor()
rfModel_PS_Q_DAY.fit(df_PS_Q_DAYWith[independentVariables], df_PS_Q_DAYWith['PS_Q_DAY'])

generatedPS_Q_DAYValues = rfModel_PS_Q_DAY.predict(X = df_PS_Q_DAYWithout[independentVariables])

print(generatedPS_Q_DAYValues)
df_PS_Q_DAYWithout['PS_Q_DAY'] = generatedPS_Q_DAYValues.astype(float)
data = df_PS_Q_DAYWith.append(df_PS_Q_DAYWithout)

data.reset_index(inplace=True)
data.drop('index',inplace=True,axis=1)

data.set_index('Date', inplace=True)
data = data.sort_index(ascending=True)

filled_df['PS_Q_DAY'] = data['PS_Q_DAY']

[1025.4126]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
percent_missing = filled_df.isnull().sum() * 100 / len(filled_df)
missing_value_df = pd.DataFrame({'percent_missing': percent_missing})
missing_value_df

,percent_missing
PS_Q_DAY,0.000000
TPS_Q1_DAY,0.117509
TWAS_DAF_QIN_DAY,0.000000
DIGESTED_SLUDGE_QOUT_DAY,0.000000
BIOGAS_PRODUCTION_Q_DAY,0.000000
DIG_SLUDGE_DEWATER_DS_AFTER_DEWATER_3_PER_WEEK,6.580494


In [ ]:
# TPS_Q1_DAY
df_TPS_Q1_DAY = prepdf_nonpca.drop(columns=['DIG_SLUDGE_DEWATER_DS_AFTER_DEWATER_3_PER_WEEK', 'PS_Q_DAY'])

df_TPS_Q1_DAYWith = df_TPS_Q1_DAY[pd.isnull(df_TPS_Q1_DAY['TPS_Q1_DAY']) == False]
df_TPS_Q1_DAYWithout = df_TPS_Q1_DAY[pd.isnull(df_TPS_Q1_DAY['TPS_Q1_DAY'])]

variables = ['TPS_Q1_DAY', 'TWAS_DAF_QIN_DAY', 'DIGESTED_SLUDGE_QOUT_DAY', 'BIOGAS_PRODUCTION_Q_DAY']

independentVariables = ['TWAS_DAF_QIN_DAY', 'DIGESTED_SLUDGE_QOUT_DAY', 'BIOGAS_PRODUCTION_Q_DAY']

rfModel_TPS_Q1_DAY = RandomForestRegressor()
rfModel_TPS_Q1_DAY.fit(df_TPS_Q1_DAYWith[independentVariables], df_TPS_Q1_DAYWith['TPS_Q1_DAY'])

generatedTPS_Q1_DAYValues = rfModel_TPS_Q1_DAY.predict(X = df_TPS_Q1_DAYWithout[independentVariables])

print(generatedTPS_Q1_DAYValues)
df_TPS_Q1_DAYWithout['TPS_Q1_DAY'] = generatedTPS_Q1_DAYValues.astype(float)
data = df_TPS_Q1_DAYWith.append(df_TPS_Q1_DAYWithout)

data.reset_index(inplace=True)
data.drop('index',inplace=True,axis=1)

data.set_index('Date', inplace=True)
data = data.sort_index(ascending=True)

filled_df['TPS_Q1_DAY'] = data['TPS_Q1_DAY']
#data.TPS_Q1_DAY
percent_missing = filled_df.isnull().sum() * 100 / len(filled_df)
missing_value_df = pd.DataFrame({'percent_missing': percent_missing})
missing_value_df

[199.5]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,percent_missing
PS_Q_DAY,0.000000
TPS_Q1_DAY,0.000000
TWAS_DAF_QIN_DAY,0.000000
DIGESTED_SLUDGE_QOUT_DAY,0.000000
BIOGAS_PRODUCTION_Q_DAY,0.000000
DIG_SLUDGE_DEWATER_DS_AFTER_DEWATER_3_PER_WEEK,6.580494


In [ ]:
# DIG_SLUDGE_DEWATER_DS_AFTER_DEWATER_3_PER_WEEK
df_DIG_SLUDGE_DEWATER_DS_AFTER_DEWATER_3_PER_WEEK = filled_df.copy()#.drop(columns=['DIGESTED_SLUDGE_VS_3_PER_WEEK', 'TPS_DS_DAY', 'PS_VS_DAY'])

df_DIG_SLUDGE_DEWATER_DS_AFTER_DEWATER_3_PER_WEEKWith = df_DIG_SLUDGE_DEWATER_DS_AFTER_DEWATER_3_PER_WEEK[pd.isnull(df_DIG_SLUDGE_DEWATER_DS_AFTER_DEWATER_3_PER_WEEK['DIG_SLUDGE_DEWATER_DS_AFTER_DEWATER_3_PER_WEEK']) == False]
df_DIG_SLUDGE_DEWATER_DS_AFTER_DEWATER_3_PER_WEEKWithout = df_DIG_SLUDGE_DEWATER_DS_AFTER_DEWATER_3_PER_WEEK[pd.isnull(df_DIG_SLUDGE_DEWATER_DS_AFTER_DEWATER_3_PER_WEEK['DIG_SLUDGE_DEWATER_DS_AFTER_DEWATER_3_PER_WEEK'])]

variables = ['PS_Q_DAY', 'TWAS_DAF_QIN_DAY', 'DIGESTED_SLUDGE_QOUT_DAY', 'BIOGAS_PRODUCTION_Q_DAY','TPS_Q1_DAY' ,'DIG_SLUDGE_DEWATER_DS_AFTER_DEWATER_3_PER_WEEK']

independentVariables = ['PS_Q_DAY', 'TWAS_DAF_QIN_DAY', 'DIGESTED_SLUDGE_QOUT_DAY', 'BIOGAS_PRODUCTION_Q_DAY','TPS_Q1_DAY' ]

rfModel_DIG_SLUDGE_DEWATER_DS_AFTER_DEWATER_3_PER_WEEK = RandomForestRegressor()
rfModel_DIG_SLUDGE_DEWATER_DS_AFTER_DEWATER_3_PER_WEEK.fit(df_DIG_SLUDGE_DEWATER_DS_AFTER_DEWATER_3_PER_WEEKWith[independentVariables], df_DIG_SLUDGE_DEWATER_DS_AFTER_DEWATER_3_PER_WEEKWith['DIG_SLUDGE_DEWATER_DS_AFTER_DEWATER_3_PER_WEEK'])

generatedDIG_SLUDGE_DEWATER_DS_AFTER_DEWATER_3_PER_WEEKValues = rfModel_DIG_SLUDGE_DEWATER_DS_AFTER_DEWATER_3_PER_WEEK.predict(X = df_DIG_SLUDGE_DEWATER_DS_AFTER_DEWATER_3_PER_WEEKWithout[independentVariables])

print(generatedDIG_SLUDGE_DEWATER_DS_AFTER_DEWATER_3_PER_WEEKValues)
df_DIG_SLUDGE_DEWATER_DS_AFTER_DEWATER_3_PER_WEEKWithout['DIG_SLUDGE_DEWATER_DS_AFTER_DEWATER_3_PER_WEEK'] = generatedDIG_SLUDGE_DEWATER_DS_AFTER_DEWATER_3_PER_WEEKValues.astype(float)
data = df_DIG_SLUDGE_DEWATER_DS_AFTER_DEWATER_3_PER_WEEKWith.append(df_DIG_SLUDGE_DEWATER_DS_AFTER_DEWATER_3_PER_WEEKWithout)

data.reset_index(inplace=True)


data.set_index('Date', inplace=True)
data = data.sort_index(ascending=True)

filled_df['DIG_SLUDGE_DEWATER_DS_AFTER_DEWATER_3_PER_WEEK'] = data['DIG_SLUDGE_DEWATER_DS_AFTER_DEWATER_3_PER_WEEK']
percent_missing = filled_df.isnull().sum() * 100 / len(filled_df)
missing_value_df = pd.DataFrame({'percent_missing': percent_missing})
missing_value_df

[31.9183 31.761  31.3773 30.925  32.394  31.546  30.229  31.14   31.714
 32.369  31.8368 30.98   30.8668 30.356  30.9054 30.854  31.577  31.233
 31.098  30.9843 32.489  30.1368 31.2437 30.8741 30.781  30.4795 30.6596
 30.8422 30.8366 30.3474 29.8472 30.3987 31.6196 30.3597 30.676  30.9358
 30.4189 32.1584 30.109  30.1493 31.0941 31.7284 30.8812 30.9709 30.5735
 30.6382 30.1464 29.7768 31.3722 30.4317 30.4368 30.8445 30.4933 29.7684
 30.2479 30.2315]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,percent_missing
PS_Q_DAY,0.0
TPS_Q1_DAY,0.0
TWAS_DAF_QIN_DAY,0.0
DIGESTED_SLUDGE_QOUT_DAY,0.0
BIOGAS_PRODUCTION_Q_DAY,0.0
DIG_SLUDGE_DEWATER_DS_AFTER_DEWATER_3_PER_WEEK,0.0


In [ ]:
#PCA NOT IN USE
# EXPLAIN WHY
pc = PCA(data=prepdf,ncomp=1, missing='fill-em')
temp=pc._adjusted_data

TypeError: ignored

In [ ]:
filled_df.head()

,PS_Q_DAY,TPS_Q1_DAY,TWAS_DAF_QIN_DAY,DIGESTED_SLUDGE_QOUT_DAY,BIOGAS_PRODUCTION_Q_DAY,DIG_SLUDGE_DEWATER_DS_AFTER_DEWATER_3_PER_WEEK
Date,,,,,,
2012-01-01,1197.0,199.5,242.5,414,5.064,32.1
2012-01-02,1197.0,199.5,165.5,392,5.293,32.1
2012-01-03,1197.0,199.5,241.5,428,5.609,32.1
2012-01-04,1197.0,170.0,194.0,354,4.470,31.8
2012-01-05,1197.0,199.5,193.5,383,2.848,31.4


In [ ]:
filled_df.to_csv("viable_dataset.csv")

In [4]:
df = pd.read_csv("viable_dataset.csv",index_col=0)

In [5]:
df.head()

,PS_Q_DAY,TPS_Q1_DAY,TWAS_DAF_QIN_DAY,DIGESTED_SLUDGE_QOUT_DAY,BIOGAS_PRODUCTION_Q_DAY,DIG_SLUDGE_DEWATER_DS_AFTER_DEWATER_3_PER_WEEK
Date,,,,,,
2012-01-01,1197.0,199.5,242.5,414,5.064,32.1
2012-01-02,1197.0,199.5,165.5,392,5.293,32.1
2012-01-03,1197.0,199.5,241.5,428,5.609,32.1
2012-01-04,1197.0,170.0,194.0,354,4.470,31.8
2012-01-05,1197.0,199.5,193.5,383,2.848,31.4


In [6]:
from statsmodels.tsa.stattools import adfuller
# if test_statistic greater than critical value, data is not stationary
def adf_test(timeseries):
    #Perform Dickey-Fuller test:
    dftest = adfuller(timeseries, autolag='AIC')
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
       dfoutput['Critical Value (%s)'%key] = value
    return dfoutput

In [8]:
adf_test(df.TWAS_DAF_QIN_DAY)# Stationary

Test Statistic                -8.049189e+00
p-value                        1.760760e-12
#Lags Used                     4.000000e+00
Number of Observations Used    8.460000e+02
Critical Value (1%)           -3.438103e+00
Critical Value (5%)           -2.864962e+00
Critical Value (10%)          -2.568592e+00
dtype: float64

In [9]:
adf_test(df.PS_Q_DAY)# NOT STationary

Test Statistic                  -2.334147
p-value                          0.161191
#Lags Used                      21.000000
Number of Observations Used    829.000000
Critical Value (1%)             -3.438263
Critical Value (5%)             -2.865033
Critical Value (10%)            -2.568630
dtype: float64

In [13]:
adf_test(df.TPS_Q1_DAY)#Not Stationary

Test Statistic                  -3.160746
p-value                          0.022372
#Lags Used                      21.000000
Number of Observations Used    829.000000
Critical Value (1%)             -3.438263
Critical Value (5%)             -2.865033
Critical Value (10%)            -2.568630
dtype: float64

In [11]:
adf_test(df.DIGESTED_SLUDGE_QOUT_DAY)#Not Stationary

Test Statistic                  -2.334147
p-value                          0.161191
#Lags Used                      21.000000
Number of Observations Used    829.000000
Critical Value (1%)             -3.438263
Critical Value (5%)             -2.865033
Critical Value (10%)            -2.568630
dtype: float64

In [12]:
adf_test(df.BIOGAS_PRODUCTION_Q_DAY)# Not Stationary

Test Statistic                  -2.334147
p-value                          0.161191
#Lags Used                      21.000000
Number of Observations Used    829.000000
Critical Value (1%)             -3.438263
Critical Value (5%)             -2.865033
Critical Value (10%)            -2.568630
dtype: float64

In [14]:
adf_test(df.DIG_SLUDGE_DEWATER_DS_AFTER_DEWATER_3_PER_WEEK)# Stationary

Test Statistic                -1.066449e+01
p-value                        4.322324e-19
#Lags Used                     3.000000e+00
Number of Observations Used    8.470000e+02
Critical Value (1%)           -3.438094e+00
Critical Value (5%)           -2.864958e+00
Critical Value (10%)          -2.568590e+00
dtype: float64